In [66]:
import h5py
import numpy


def extract_spikes(hd5_file, neuron_num=0):
    """Extracts the spiking data from the hdf5 file. Returns an array of
    spike times.

    Keyword arguments:
    neuron_num -- the index of the neuron you would like to access.
    """
    with h5py.File(hd5_file, "r+") as f:
        neuron_list = f['NF']['value']['neurons']['value']
        if len(neuron_list) <= 10:
            neuron_str = "_" + str(neuron_num)
        else:
            neuron_str = "_" + "0" * (2 - len(str(neuron_num))) + str(neuron_num)
        timestamps = numpy.array(neuron_list[neuron_str]['value']['timestamps']['value'][0])
    return(timestamps)


def extract_events(hd5_file):
    """Extracts the timestamps of the events stored in the hdf5 file."""
    events = {}
    with h5py.File(hd5_file, "r+") as f:
        event_list = f['NF']['value']['events']['value']
        for key in event_list.keys():
            if key == 'dims':
                continue
            name = event_list[key]['value']['name']['value']
            # The hdf5 that results file contains the names not as strings
            # but as an array of integers which code for the string in
            # ASCII format.
            name_str = ''.join(map(chr, name))
            try:
                timestamps = numpy.array(event_list[key]['value']['timestamps']['value'][0])
            except:
                timestamps = numpy.array([], dtype='float64')
            events[name_str] = timestamps
    return(events)


def convert_data(new_file_name, spikes_file, neuron_num, events_file, conversion_table):
    spikes = extract_spikes(spikes_file, neuron_num=neuron_num)
    events = extract_events(events_file)
    animal_data = {new_k: events[k] for new_k, k in conversion_table.items() if k in events.keys()}
    animal_data['spikes'] = spikes

    with h5py.File(new_file_name + ".hd5", "w") as h5_file:
        for name, data in animal_data.items():
            print(name)
            h5_file.create_dataset(str(name), data=data)

    
VAR_DEFAULT_NEW_P5 = {'T': 'target', 'F' : 'foils', 'T+' : 'correct_target', 'F+': 'correct_rejects', 'NPT': 'npt', 'NPF': 'npf', .5 : 'DIO_00032', 1 : 'DIO_02048', 2 : 'DIO_04096', 4 : 'DIO_16384', 8 : 'DIO_00001', 16 : 'target', 32 : 'DIO_32768'}

VAR_AC_08282013 = {'F' : 'Foil', 'T' : 'Target', 'T+': 'correct_target', 'T-' : "incorrect_target", 'NPT' : 'Hits'}

VAR_DEFAULT_NEW = {'T': 'target', 'F' : 'foils', 'T+' : 'correct_target', 'F+': 'correct_rejects', 'NPT': 'npt', 'NPF': 'npf', .5 : 'DIO_00032', 1 : 'DIO_02048', 2 : 'DIO_04096', 4 : 'target', 8 : 'DIO_00001', 16 : 'DIO_16384', 32 : 'DIO_32768'}


In [57]:
convert_data("CR_AC_02272014_0", "Feb272013_003_P5_filtCentr001-02.hd5", 0, "feb 27003.hd5", VAR_DEFAULT_NEW_P5)



T
F
T+
F+
NPT
NPF
0.5
1
2
4
8
16
32
spikes


In [67]:
convert_data("NCR_AC_08282013_3", "./08282013_beh004threshCentrOF-01_neuron&events.hd5", 3, "./08282013_behavior004.hd5", VAR_DEFAULT_NEW)



T
F
T+
F+
NPT
NPF
4
spikes


In [68]:
convert_data("CR-AC_09192013_8", "./sept 19001-resort.hd5", 8, "./sept 19001.hd5", VAR_DEFAULT_NEW)



T
F
T+
F+
NPT
NPF
1
2
4
16
32
spikes
